In [ ]:
from datasets import load_dataset

#ethics_co= load_dataset("hendrycks/ethics", "commonsense") # 0이면 긍정적, 1이면 부정적 (얘는 윤리철학 측면에서 크게 중요해보이지 않아서 사용 x)
ethics_de= load_dataset("hendrycks/ethics", "deontology") # 1이 타당 (senario와 excuse 같은 행끼리 합쳐)
ethics_ju= load_dataset("hendrycks/ethics", "justice") # 1이 타당한 맥락 (senario만 사용)
#ethics_ut= load_dataset("hendrycks/ethics", "utilitarianism") # 이분법테스트 x , 선호도에 따른 분류 (이 데이터는 사용 x)
ethics_vi= load_dataset("hendrycks/ethics", "virtue") # 0이 악덕, 1이 덕 같긴 한데 애매해보임 (정제할 땐 레이블 무관 전부)

In [54]:
deontology = [] # 0이면 타당 x, 1이면 타당
settypes = ['train', 'validation', 'test']

for settype in settypes : 
    for idx in range(len(ethics_de[settype])) : 
        if ethics_de[settype][idx]['label'] == 0 : continue
        
        tmp = []
        tmp.append(ethics_de[settype][idx]['scenario'])
        tmp.append(ethics_de[settype][idx]['excuse'])
        tmp = ' '.join(tmp)
        deontology.append(tmp)


In [55]:
len(deontology)

13339

In [56]:
justice = [] # 1이면 타당
settypes = ['train', 'validation', 'test']

for settype in settypes : 
    for idx in range(len(ethics_ju[settype])) : 
        if ethics_ju[settype][idx]['label'] == 0 : continue
        justice.append(ethics_ju[settype][idx]['scenario'])



In [57]:
len(justice)

14234

In [58]:
virtue = [] # 1이면 타당
settypes = ['train', 'validation', 'test']

for settype in settypes : 
    for idx in range(len(ethics_vi[settype])) : 
        if ethics_vi[settype][idx]['label'] == 0 : continue
        
        tmp = []
        tmp.append(ethics_vi[settype][idx]['scenario'])
        tmp = tmp[0].replace('[SEP]', '==')
        virtue.append(tmp)


In [59]:
len(virtue)

4366